## Load data

In [1]:
plaacFasta = '../positiveExamples/yeastPrion.fasta'
negativeFasta = '../negativeExamples/negative.fasta'
positiveDataset = '../positiveExamples/positiveDataset.csv'
negativeDataset = '../negativeExamples/negativeDataset.csv'
destination_folder = "./output"

def fastaToCSV(fastaPath, csvPath, prionClassification = 0, amyloidClassification = 0):
    # Read in FASTA
    with open(fastaPath, 'r') as file:
        lines = ["header,sequence,prion,amyloid"]
        lines_i = file.readlines()

        seq = ''
        index = 0
        while index < len(lines_i):
            if lines_i[index][0] == '>':
                'Fasta head line'
                # append sequence & prion classification
                if seq: lines.append(seq + ',' + str(prionClassification) + ',' + str(amyloidClassification))
                seq = ''
                # remove FASTA syntactic sugar
                seq_id = lines_i[index].strip().replace(',', '').replace('>', '')
                # remove uniprot metadata using pipe & equal sign delimiter
                seq_id = " ".join((seq_id.split(" "))[1:]) if "|" in seq_id else seq_id
                lastHeaderIndex = seq_id.index("=") - 3 if ('=' in seq_id) else len(seq_id)
                lines.append('\n' + seq_id[:lastHeaderIndex] + ',')
            else:
                'Sequence line'
                seq += lines_i[index].strip()
                if (index == len(lines_i) - 1): lines.append(seq + ',' + str(prionClassification) + ',' + str(amyloidClassification))
            index += 1
        lines.append("\n")
        file.close()

    # Output CSV file
    with open(csvPath, 'w') as file:
        file.writelines(lines)
        file.close()
    return

# negative
fastaToCSV(negativeFasta, negativeDataset)
# PLAAC prions
fastaToCSV(plaacFasta, positiveDataset, 1, 1)

In [2]:
import json 
import csv 

amyproDataset = '../positiveExamples/amyproAmyloids.json'

with open(amyproDataset, encoding = 'utf-8-sig') as json_file: 
    data = json.load(json_file) 
    lines = []
    for amyloid in data:
        lines.append(amyloid["protein_name"].replace(',', '') + (' [' + amyloid["uniprot_start"] + '-' + amyloid["uniprot_end"] + ']' if amyloid["uniprot_start"] != "" else "")+ ',')
        lines.append(amyloid["sequence"] + ',')
        lines.append(str(amyloid["prion_domain"]) + ',')
        # positive amyloid classification
        lines.append('1' + '\n')
    json_file.close()

# Append to CSV file
with open(positiveDataset, 'a') as file:
    file.writelines(lines)
    file.close()

In [3]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split, KFold

train_validation_ratio = 0.85
train_test_ratio = 0.7
df_train, df_test = pd.DataFrame(), pd.DataFrame()

def generateDataFrames(randomSeed):
    # Read raw data & combine 
    df_positive = pd.read_csv(positiveDataset).applymap(lambda x: x.strip() if type(x)==str else x)
    df_negative = pd.read_csv(negativeDataset).applymap(lambda x: x.strip() if type(x)==str else x)
    df_full = pd.concat([df_positive, df_negative], ignore_index=True)

    # Generate in-memory index for headers & compress dataframe
    datasetIndex = {}
    for index in range(len(df_full.index)):
        datasetIndex[index] = df_full.at[index, 'header']
        df_full.at[index, 'header'] = index

    # Split amyloid class
    df_notAmyloid = df_full[df_full['amyloid'] == 0]
    df_amyloid = df_full[df_full['amyloid'] == 1]

    # Train-validation split
    df_amyloid_trainSuperset, df_amyloid_validation = train_test_split(df_amyloid, train_size = train_validation_ratio, random_state = randomSeed)
    df_notAmyloid_trainSuperset, df_notAmyloid_validation  = train_test_split(df_notAmyloid, train_size = train_validation_ratio, random_state = randomSeed)

    # Train-test split
    df_amyloid_train, df_amyloid_test = train_test_split(df_amyloid_trainSuperset, train_size = train_test_ratio, random_state = randomSeed)
    df_notAmyloid_train, df_notAmyloid_test = train_test_split(df_notAmyloid_trainSuperset, train_size = train_test_ratio, random_state = randomSeed)

    # Concatenate splits of different labels into training, testing & validation sets
    df_train = pd.concat([df_amyloid_train, df_notAmyloid_train], ignore_index=True, sort=False)
    df_test = pd.concat([df_amyloid_test, df_notAmyloid_test], ignore_index=True, sort=False)
    df_valid = pd.concat([df_amyloid_validation, df_notAmyloid_validation], ignore_index=True, sort=False)
    df_full = pd.concat([df_train, df_test], ignore_index=True, sort=False)

    # Write preprocessed data
    df_train.to_csv('train.csv', index=False)
    df_test.to_csv('test.csv', index=False)
    df_valid.to_csv('valid.csv', index=False)
    df_full.to_csv('full.csv', index=False)

    return datasetIndex

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Tokenize peptide sequences by splitting into individual amino acids
def split(sequence):
    return [char for char in sequence]

import pandas as pd
from sklearn.model_selection import train_test_split

datasetIndex = generateDataFrames(0)
# Load CSV data
df_train = pd.concat([pd.read_csv('train.csv'), pd.read_csv('test.csv')], ignore_index=True)

# Generate vocabulary
tokenizer = get_tokenizer(split)
vocab = build_vocab_from_iterator(map(tokenizer, df_train['sequence']))

In [5]:
import torch
torch.save(vocab, 'vocab.pt')

In [ ]:
from torch.utils.data import Dataset

class AmyloidDataset(Dataset):
    def __init__(self, dataframe, vocab, tokenizer):
        self.data = dataframe
        self.vocab = vocab
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sequence = self.data.iloc[idx]["sequence"]
        sequence = [self.vocab[token] for token in self.tokenizer(sequence)]
        header = self.data.iloc[idx]["header"]

        prion = self.data.iloc[idx]["prion"] if "prion" in self.data.columns else None
        amyloid = (
            self.data.iloc[idx]["amyloid"] if "amyloid" in self.data.columns else None
        )

        return header, sequence, prion, amyloid

## Model

In [5]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class LitGRU(pl.LightningModule):
    def __init__(self, vocab, dimension=64, sequenceDepth=64, dropoutWithinLayers=0.5, dropoutOutput=0.5, learningRate=1e-3, amyloid=True, lossFunc=nn.SmoothL1Loss()):
        super(LitGRU, self).__init__()
        self.embedding = nn.Embedding(len(vocab), sequenceDepth)
        self.dimension = dimension
        self.gru = nn.GRU(input_size=sequenceDepth,
                            hidden_size=dimension,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True,
                            dropout=dropoutWithinLayers)
        self.dropOnOutput = nn.Dropout(p=dropoutOutput)
        self.learningRate = learningRate
        self.fc = nn.Linear(2 * dimension, 1)
        self.loss_func = lossFunc
        self.amyloid = amyloid
        self.save_hyperparameters()

    def forward(self, text, text_len):
        text_emb = self.embedding(text)
        packed_input = pack_padded_sequence(text_emb, text_len.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, _ = self.gru(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        out_forward = output[range(len(output)), text_len - 1, :self.dimension]
        out_reverse = output[:, 0, self.dimension:]
        out_reduced = torch.cat((out_forward, out_reverse), 1)
        text_fea = self.dropOnOutput(out_reduced)
        text_fea = self.fc(text_fea)
        text_fea = torch.squeeze(text_fea, 1)
        text_out = torch.sigmoid(text_fea)
        return text_out

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learningRate)
        return optimizer

    def training_step(self, batch, batch_idx):
        header, sequence, sequence_length, prionLabel, amyloidLabel = batch
        output = self(sequence, sequence_length)
        loss = self.loss_func(output, prionLabel if not self.hparams['amyloid'] else amyloidLabel)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        header, sequence, sequence_length, prionLabel, amyloidLabel = batch
        output = self(sequence, sequence_length)
        val_loss = self.loss_func(output, prionLabel if not self.hparams['amyloid'] else amyloidLabel)
        self.log("val_loss", val_loss, prog_bar=True)
        return {'val_loss': val_loss}


    def test_step(self, batch, batch_idx):
        header, sequence, sequence_length, prionLabel, amyloidLabel = batch
        output = self(sequence, sequence_length)
        test_loss = self.loss_func(output, prionLabel if not self.hparams['amyloid'] else amyloidLabel)
        self.log("test_loss", test_loss, prog_bar=True)
        return {'test_loss': test_loss}


In [6]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    headers, sequences, prions, amyloids = zip(*batch)

    # Compute lengths of sequences
    sequence_lengths = torch.tensor([len(seq) for seq in sequences])

    # Pad sequences and convert to tensor
    sequences_padded = pad_sequence([torch.tensor(seq) for seq in sequences], batch_first=True)

    # Convert targets to tensor
    prions = torch.tensor(prions).float()
    amyloids = torch.tensor(amyloids).float()

    return headers, sequences_padded, sequence_lengths, prions, amyloids

## Tune parameters

In [16]:
wandb_kwargs = {"project": "agGRU-Check", "group": "tuning1"}

In [7]:
from pytorch_lightning import Trainer, loggers
from deployment.backend.dataset import AmyloidDataset
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import KFold
from tqdm import tqdm
import numpy as np
import os

kfold = KFold(n_splits=10, shuffle=True)

In [17]:
def objective(trial):
    # define hyperparameters
    dimension = trial.suggest_int('dimension', 32, 256)
    sequenceDepth = trial.suggest_int('sequenceDepth', 16, 128)
    dropoutWithinLayers = trial.suggest_uniform('dropoutWithinLayers', 0.1, 0.6)
    dropoutOutput = trial.suggest_uniform('dropoutOutput', 0.1, 0.6)
    learningRate = trial.suggest_loguniform('learningRate', 1e-5, 1e-1)
    
    # Initialize Weights & Biases logger
    wandb_logger = loggers.WandbLogger(**wandb_kwargs)

    losses = []
    for fold, (train_ids, test_ids) in tqdm(enumerate(kfold.split(df_train)), unit='fold'):
        # Split data
        train_data, test_data = df_train.iloc[train_ids], df_train.iloc[test_ids]

        # Create datasets
        train_dataset = AmyloidDataset(train_data, vocab, tokenizer)
        test_dataset = AmyloidDataset(test_data, vocab, tokenizer)

        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=os.cpu_count())
        test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn, num_workers=os.cpu_count())
        
        model = LitGRU(vocab, dimension=dimension, sequenceDepth=sequenceDepth, dropoutWithinLayers=dropoutWithinLayers, dropoutOutput=dropoutOutput, learningRate=learningRate)

        # Pass the logger to the Trainer
        trainer = Trainer(max_epochs=5, enable_progress_bar=True, enable_model_summary=True, log_every_n_steps=2, logger=wandb_logger)
        trainer.fit(model, train_loader, test_loader)
        
        # Save loss for each fold
        losses.append(trainer.callback_metrics['val_loss'].item())
    
    # Return the average validation loss across all folds
    return np.mean(losses)


In [18]:
import optuna
from optuna.integration import WeightsAndBiasesCallback

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, callbacks=[WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)])

[I 2023-07-15 20:51:17,140] A new study created in memory with name: no-name-e585f142-e2f9-476b-bdf7-6e1b9cdca2fd
/tmp/ipykernel_310438/3399104407.py:8: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  study.optimize(objective, n_trials=100, callbacks=[WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)])


/tmp/ipykernel_310438/1021991541.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropoutWithinLayers = trial.suggest_uniform('dropoutWithinLayers', 0.1, 0.6)
/tmp/ipykernel_310438/1021991541.py:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropoutOutput = trial.suggest_uniform('dropoutOutput', 0.1, 0.6)
/tmp/ipykernel_310438/1021991541.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learningRate = trial.suggest_loguniform('learningRate', 1e-5, 1e-1)
/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pytorch_lig

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
10fold [04:43, 28.39s/fold]
[I 2023-07-16 04:43:35,158] Trial 99 finished with value: 0.06869197599589824 and parameters: {'dimension': 73, 'sequenceDepth': 78, 'dropoutWithinLayers': 0.2110619450026311, 'dropoutOutput': 0.4529553802369328, 'learningRate': 0.0002271850789726301}. Best is trial 72 with value: 0.0636373370885849.


In [19]:
study.best_trial.params

{'dimension': 106,
 'sequenceDepth': 85,
 'dropoutWithinLayers': 0.31686011720357404,
 'dropoutOutput': 0.5638720207488487,
 'learningRate': 0.0003064907979479033}

## Train

In [8]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

best_params = eval("{'dimension': 106,'sequenceDepth': 85,'dropoutWithinLayers': 0.31686011720357404,'dropoutOutput': 0.5638720207488487,'learningRate': 0.0003064907979479033}")
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # or any other quantity you want to monitor
    dirpath='checkpoints',  # directory where to save model checkpoints
    filename='model-{epoch:02d}-{val_loss:.2f}',  # filename format
    save_top_k=3,  # number of best models to save
    mode='min',  # mode for the monitor quantity
)
for fold, (train_ids, test_ids) in tqdm(enumerate(kfold.split(df_train)), unit='fold'):
    # Split data
    train_data, test_data = df_train.iloc[train_ids], df_train.iloc[test_ids]

    # Create datasets
    train_dataset = AmyloidDataset(train_data, vocab, tokenizer)
    test_dataset = AmyloidDataset(test_data, vocab, tokenizer)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=os.cpu_count())
    test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn, num_workers=os.cpu_count())
    
    model = LitGRU(vocab, dimension=best_params['dimension'], sequenceDepth=best_params['sequenceDepth'], dropoutWithinLayers=best_params['dropoutWithinLayers'], dropoutOutput=best_params['dropoutOutput'], learningRate=best_params['learningRate'])
    trainer = Trainer(max_epochs=5 , enable_progress_bar=True, enable_model_summary=True, log_every_n_steps=2, callbacks=[checkpoint_callback])
    trainer.fit(model, train_loader, test_loader)


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-07-16 11:54:11.761760: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 11:54:11.784690: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 11:54:11.784933: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 11:54:12.185578: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-16 11:54:12.807047: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning N

0fold [00:00, ?fold/s]/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'vocab' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['vocab'])`.
  rank_zero_warn(
/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'lossFunc' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['lossFunc'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-07-16 11:54:14.150169: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-cr

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
10fold [04:21, 26.10s/fold]
